## 7. 퍼셉트론과 인공지능의 시작

In [9]:
import numpy as np
import random

### 7.1 인공지능의 시작을 알린 퍼셉트론

+ 퍼셉트론(perceptron)
+ 가중합(weighted sum)이란 입력 값과 가중치를 모두 곱한 후 바이어스를 더한 값을 의미

### 7.2 퍼셉트론의 과제

+ 퍼셉트론이나 아달라인은 모두 2차원 평면상에 직선을 긋는 것만 가능. 경우에 따라 선을 아무리 그어도 해결되지 않는 상황 예시

### 7.3 XOR 문제

+ XOR(exclusive OR)

## 8. 다층 퍼셉트론

### 8.1 다층 퍼셉트론의 등장

### 8.2 다층 퍼셉트론의 설계

In [3]:
w11 = np.array([-2, -2])
w12 = np.array([2, 2])
w2 = np.array([1, 1])

b1, b2, b3 = 3, -1, -1

In [4]:
def MLP(x, w, b):
    y = np.sum(w * x) + b
    
    if y <= 0:
        return 0
    else:
        return 1

In [5]:
# NAND 게이트
def NAND(x1, x2):
    return MLP(np.array([x1, x2]), w11, b1)

# OR 게이트
def OR(x1, x2):
    return MLP(np.array([x1, x2]), w12, b2)

# AND 게이트
def AND(x1, x2):
    return MLP(np.array([x1, x2]), w2, b3)

# XOR 게이트
def XOR(x1, x2):
    return AND(NAND(x1, x2), OR(x1, x2))

In [7]:
for x in [(0, 0), (1, 0), (0, 1), (1, 1)]:
    y = XOR(x[0], x[1])
    print('입력 값: ' + str(x) +' // '+ '출력값:' + str(y))

입력 값: (0, 0) // 출력값:0
입력 값: (1, 0) // 출력값:1
입력 값: (0, 1) // 출력값:1
입력 값: (1, 1) // 출력값:0


In [8]:
# 가중치와 바이어스
w11 = np.array([-2, -2])
w12 = np.array([2, 2])
w2 = np.array([1, 1])

b1, b2, b3 = 3, -1, -1

# 퍼셉트론
def MLP(x, w, b):
    y = np.sum(w * x) + b
    if y <= 0:
        return 0
    else:
        return 1
    
# NAND 게이트
def NAND(x1, x2):
    return MLP(np.array([x1, x2]), w11, b1)

# OR 게이트
def OR(x1, x2):
    return MLP(np.array([x1, x2]), w12, b2)

# AND 게이트
def AND(x1, x2):
    return MLP(np.array([x1, x2]), w2, b3)

# XOR 게이트
def XOR(x1, x2):
    return AND(NAND(x1, x2), OR(x1, x2))

# x1 값, x2 값을 번갈아 대입하며 최종 값 출력
for x in [(0, 0), (1, 0), (0, 1), (1, 1)]:
    y = XOR(x[0], x[1])
    print('입력 값: ' + str(x) + ' // ' + '출력 값: ' + str(y))

입력 값: (0, 0) // 출력 값: 0
입력 값: (1, 0) // 출력 값: 1
입력 값: (0, 1) // 출력 값: 1
입력 값: (1, 1) // 출력 값: 0


## 9. 오차 역전파에서 딥러닝으로

In [1]:
# 입력 값 및 타깃 값
data = [
    [[0, 0], [0]],
    [[0, 1], [1]],
    [[1, 0], [1]],
    [[1, 1], [0]]
]

# 실행 횟수(iterations), 학습률(lr), 모멘텀 계수(mo) 설정
iterations = 5000
lr = .1
mo = .9

In [2]:
# 활성화 함수 -> 1. 시그모이드
# 미분할 때와 아닐 때 각각의 값
def sigmoid(x, derivative=False):
    if (derivative==True):
        return x * (1 - x)
    return 1 / (1 + np.exp(-x))

# 활성화 함수 -> 2. tanh
# tanh 함수의 미분은 1 - (활성화 함수 출력의 제곱)
def tanh(x, derivative=False):
    if (derivative==True):
        return 1 - x**2
    return np.tanh(x)

# 가중치 배열을 만드는 함수
def makeMatrix(i, j, fill=0.0):
    mat = []
    for i in range(i):
        mat.append([fill] * j)
    return mat

In [12]:
# 신경망의 실행
class NeuralNetwork:
    
    # 초깃값 지정
    def __init__(self, num_x, num_yh, num_yo, bias=1):
        
        # 입력 값(num_x), 은닉층의 초깃값(num_yh), 출력층의 초깃값(num_yo), 바이어스
        self.num_x = num_x + bias # 바이어스는 1로 설정
        self.num_yh = num_yh
        self.num_yo = num_yo
        
        # 활성화 함수 초깃값
        self.activation_input = [1.0] * self.num_x
        self.activation_hidden = [1.0] * self.num_yh
        self.activation_out = [1.0] * self.num_yo
        
        # 가중치 입력 초깃값
        self.weight_in = makeMatrix(self.num_x, self.num_yh)
        for i in range(self.num_x):
            for j in range(self.num_yh):
                self.weight_in[i][j] = random.random()
                
        # 가중치 출력 초깃값
        self.weight_out = makeMatrix(self.num_yh, self.num_yo)
        for j in range(self.num_yh):
            for k in range(self.num_yo):
                self.weight_out[j][k] = random.random()
                
        # 모멘텀 SGD를 위한 이전 가중치 초깃값
        self.gradient_in = makeMatrix(self.num_x, self.num_yh)
        self.gradient_out = makeMatrix(self.num_yh, self.num_yo)
        
    # 업데이트 함수
    def update(self, inputs):

        # 입력층의 활성화 함수
        for i in range(self.num_x - 1):
            self.activation_input[i] = inputs[i]

        # 은닉층의 활성화 함수
        for j in range(self.num_yh):
            sum = 0.0
            for i in range(self.num_x):
                sum = sum + self.activation_input[i] * self.weight_in[i][j]

            # 시그모이드와 tanh 중에서 활성화 함수 선택
            self.activation_hidden[j] = tanh(sum, False)


        # 출력층의 활성화 함수
        for k in range(self.num_yo):
            sum = 0.0
            for j in range(self.num_yh):
                sum = sum + self.activation_hidden[j] * self.weight_out[j][k]

            # 시그모이드와 tanh 중에서 활성화 함수 선택
            self.activation_out[k] = tanh(sum, False)

        return self.activation_out[:]
    
    # 역전파 실행
    def backPropagate(self, target):

        # 델타 출력 계산
        output_deltas = [0.0] * self.num_yo
        for k in range(self.num_yo):
            error = targets[k] - self.activation_out[k]

            # 시그모이드와 tanh 중에서 활성화 함수 선택, 미분 적용
            output_deltas[k] = tanh(self.activation_out[k], True) * error

        # 은닉 노드와 오차 함수
        hidden_deltas = [0.0] * self.num_yh
        for j in range(self.num_yh):
            error = 0.0
            for k in range(self.num_yo):
                error = error + output_deltas[k] * self.weight_out[j][k]
            # 시그모이드와 tanh 중에서 활성화 함수 선택, 미분 적용
            hidden_deltas[j] = tanh(self.activation_hidden[j], True) * error

        # 출력 가중치 업데이트
        for j in range(self.num_yh):
            for k in range(self.num_yo):
                gradient = output_deltas[k] * self.activation_hidden[j]
                v = mo*self.gradient_out[j][k] - lr * gradient
                self.weight_out[j][k] += v
                self.gradient_out[j][k] = gradient

        # 입력 가중치 업데이트
        for i in range(self.num_x):
            for j in range(self.num_yh):
                gradient = hidden_deltas[j] * self.activation_input[i]
                v = mo * self.gradient_in[i][j] - lr * gradient
                self.weight_in[i][j] += v
                self.gradient_in[i][j] = gradient

        # 오차 계산(최소 제곱법)
        error = 0.0
        for k in range(len(targets)):
            error = error + 0.5 * (targets[k] - self.activation_out[k])**2
        return error
    
    # 학습 실행
    def train(self, patterns):
        for i in range(iterations):
            error = 0.0
            for p in patterns:
                inputs = p[0]
                targets = p[1]
                self.update(inputs)
                error = error + self.backPropagate(target)

            if i % 500 == 0:
                print('error: %-.5f' % error)


    # 결괏값 출력
    def result(self, patterns):
        for p in patterns:
            print('Input: %s, Predict: %s' % (p[0], self.update(p[0])))




    if __name__ == '__main__':
        # 두 개의 입력 값, 두 개의 레이어, 하나의 출력 값을 갖도록 설정
        n = NeuralNetwork(2, 2, 1)

        # 학습 실행
        n.train(data)

        # 결괏값 출력
        n.result(data)

AttributeError: 'NeuralNetwork' object has no attribute 'train'

In [5]:
# 업데이트 함수
def update(self, inputs):
    
    # 입력층의 활성화 함수
    for i in range(self.num_x - 1):
        self.activation_input[i] = inputs[i]
        
    # 은닉층의 활성화 함수
    for j in range(self.num_yh):
        sum = 0.0
        for i in range(self.num_x):
            sum = sum + self.activation_input[i] * self.weight_in[i][j]
            
        # 시그모이드와 tanh 중에서 활성화 함수 선택
        self.activation_hidden[j] = tanh(sum, False)
        
        
    # 출력층의 활성화 함수
    for k in range(self.num_yo):
        sum = 0.0
        for j in range(self.num_yh):
            sum = sum + self.activation_hidden[j] * self.weight_out[j][k]
            
        # 시그모이드와 tanh 중에서 활성화 함수 선택
        self.activation_out[k] = tanh(sum, False)
    
    return self.activation_out[:]

In [6]:
# 역전파 실행
def backPropagate(self, target):
    
    # 델타 출력 계산
    output_deltas = [0.0] * self.num_yo
    for k in range(self.num_yo):
        error = targets[k] - self.activation_out[k]
        
        # 시그모이드와 tanh 중에서 활성화 함수 선택, 미분 적용
        output_deltas[k] = tanh(self.activation_out[k], True) * error
        
    # 은닉 노드와 오차 함수
    hidden_deltas = [0.0] * self.num_yh
    for j in range(self.num_yh):
        error = 0.0
        for k in range(self.num_yo):
            error = error + output_deltas[k] * self.weight_out[j][k]
        # 시그모이드와 tanh 중에서 활성화 함수 선택, 미분 적용
        hidden_deltas[j] = tanh(self.activation_hidden[j], True) * error
        
    # 출력 가중치 업데이트
    for j in range(self.num_yh):
        for k in range(self.num_yo):
            gradient = output_deltas[k] * self.activation_hidden[j]
            v = mo*self.gradient_out[j][k] - lr * gradient
            self.weight_out[j][k] += v
            self.gradient_out[j][k] = gradient
            
    # 입력 가중치 업데이트
    for i in range(self.num_x):
        for j in range(self.num_yh):
            gradient = hidden_deltas[j] * self.activation_input[i]
            v = mo * self.gradient_in[i][j] - lr * gradient
            self.weight_in[i][j] += v
            self.gradient_in[i][j] = gradient
            
    # 오차 계산(최소 제곱법)
    error = 0.0
    for k in range(len(targets)):
        error = error + 0.5 * (targets[k] - self.activation_out[k])**2
    return error

In [13]:
# 신경망의 실행
class NeuralNetwork:
    
    # 초깃값 지정
    def __init__(self, num_x, num_yh, num_yo, bias=1):
        
        # 입력 값(num_x), 은닉층의 초깃값(num_yh), 출력층의 초깃값(num_yo), 바이어스
        self.num_x = num_x + bias # 바이어스는 1로 설정
        self.num_yh = num_yh
        self.num_yo = num_yo
        
        # 활성화 함수 초깃값
        self.activation_input = [1.0] * self.num_x
        self.activation_hidden = [1.0] * self.num_yh
        self.activation_out = [1.0] * self.num_yo
        
        # 가중치 입력 초깃값
        self.weight_in = makeMatrix(self.num_x, self.num_yh)
        for i in range(self.num_x):
            for j in range(self.num_yh):
                self.weight_in[i][j] = random.random()
                
        # 가중치 출력 초깃값
        self.weight_out = makeMatrix(self.num_yh, self.num_yo)
        for j in range(self.num_yh):
            for k in range(self.num_yo):
                self.weight_out[j][k] = random.random()
                
        # 모멘텀 SGD를 위한 이전 가중치 초깃값
        self.gradient_in = makeMatrix(self.num_x, self.num_yh)
        self.gradient_out = makeMatrix(self.num_yh, self.num_yo)
        
    # 업데이트 함수
    def update(self, inputs):

        # 입력층의 활성화 함수
        for i in range(self.num_x - 1):
            self.activation_input[i] = inputs[i]

        # 은닉층의 활성화 함수
        for j in range(self.num_yh):
            sum = 0.0
            for i in range(self.num_x):
                sum = sum + self.activation_input[i] * self.weight_in[i][j]

            # 시그모이드와 tanh 중에서 활성화 함수 선택
            self.activation_hidden[j] = tanh(sum, False)


        # 출력층의 활성화 함수
        for k in range(self.num_yo):
            sum = 0.0
            for j in range(self.num_yh):
                sum = sum + self.activation_hidden[j] * self.weight_out[j][k]

            # 시그모이드와 tanh 중에서 활성화 함수 선택
            self.activation_out[k] = tanh(sum, False)

        return self.activation_out[:]
    
    # 역전파 실행
    def backPropagate(self, target):

        # 델타 출력 계산
        output_deltas = [0.0] * self.num_yo
        for k in range(self.num_yo):
            error = targets[k] - self.activation_out[k]

            # 시그모이드와 tanh 중에서 활성화 함수 선택, 미분 적용
            output_deltas[k] = tanh(self.activation_out[k], True) * error

        # 은닉 노드와 오차 함수
        hidden_deltas = [0.0] * self.num_yh
        for j in range(self.num_yh):
            error = 0.0
            for k in range(self.num_yo):
                error = error + output_deltas[k] * self.weight_out[j][k]
            # 시그모이드와 tanh 중에서 활성화 함수 선택, 미분 적용
            hidden_deltas[j] = tanh(self.activation_hidden[j], True) * error

        # 출력 가중치 업데이트
        for j in range(self.num_yh):
            for k in range(self.num_yo):
                gradient = output_deltas[k] * self.activation_hidden[j]
                v = mo*self.gradient_out[j][k] - lr * gradient
                self.weight_out[j][k] += v
                self.gradient_out[j][k] = gradient

        # 입력 가중치 업데이트
        for i in range(self.num_x):
            for j in range(self.num_yh):
                gradient = hidden_deltas[j] * self.activation_input[i]
                v = mo * self.gradient_in[i][j] - lr * gradient
                self.weight_in[i][j] += v
                self.gradient_in[i][j] = gradient

        # 오차 계산(최소 제곱법)
        error = 0.0
        for k in range(len(targets)):
            error = error + 0.5 * (targets[k] - self.activation_out[k])**2
        return error
    
    # 학습 실행
    def train(self, patterns):
        for i in range(iterations):
            error = 0.0
            for p in patterns:
                inputs = p[0]
                targets = p[1]
                self.update(inputs)
                error = error + self.backPropagate(target)

            if i % 500 == 0:
                print('error: %-.5f' % error)


    # 결괏값 출력
    def result(self, patterns):
        for p in patterns:
            print('Input: %s, Predict: %s' % (p[0], self.update(p[0])))




    if __name__ == '__main__':
        # 두 개의 입력 값, 두 개의 레이어, 하나의 출력 값을 갖도록 설정
        n = NeuralNetwork(2, 2, 1)

        # 학습 실행
        n.train(data)

        # 결괏값 출력
        n.result(data)

AttributeError: 'NeuralNetwork' object has no attribute 'train'

In [14]:
n = NeuralNetwork(2, 2, 1)

In [10]:
# 학습 실행
def train(self, patterns):
    for i in range(iterations):
        error = 0.0
        for p in patterns:
            inputs = p[0]
            targets = p[1]
            self.update(inputs)
            error = error + self.backPropagate(target)
        
        if i % 500 == 0:
            print('error: %-.5f' % error)
            

# 결괏값 출력
def result(self, patterns):
    for p in patterns:
        print('Input: %s, Predict: %s' % (p[0], self.update(p[0])))
        

        
        
if __name__ == '__main__':
    # 두 개의 입력 값, 두 개의 레이어, 하나의 출력 값을 갖도록 설정
    n = NeuralNetwork(2, 2, 1)
    
    # 학습 실행
    n.train(data)
    
    # 결괏값 출력
    n.result(data)

AttributeError: 'NeuralNetwork' object has no attribute 'train'

In [19]:
zz

AttributeError: 'NeuralNetwork' object has no attribute 'train'

In [20]:
import math
import random
import string

random.seed(0)

# calculate a random number where:  a <= rand < b
def rand(a, b):
    return (b-a)*random.random() + a

# Make a matrix (we could use NumPy to speed this up)
def makeMatrix(I, J, fill=0.0):
    m = []
    for i in range(I):
        m.append([fill]*J)
    return m

# our sigmoid function, tanh is a little nicer than the standard 1/(1+e^-x)
def sigmoid(x):
    return math.tanh(x)

# derivative of our sigmoid function, in terms of the output (i.e. y)
def dsigmoid(y):
    return 1.0 - y**2

class NN:
    def __init__(self, ni, nh, no):
        # number of input, hidden, and output nodes
        self.ni = ni + 1 # +1 for bias node
        self.nh = nh
        self.no = no

        # activations for nodes
        self.ai = [1.0]*self.ni
        self.ah = [1.0]*self.nh
        self.ao = [1.0]*self.no
        
        # create weights
        self.wi = makeMatrix(self.ni, self.nh)
        self.wo = makeMatrix(self.nh, self.no)
        # set them to random vaules
        for i in range(self.ni):
            for j in range(self.nh):
                self.wi[i][j] = rand(-0.2, 0.2)
        for j in range(self.nh):
            for k in range(self.no):
                self.wo[j][k] = rand(-2.0, 2.0)

        # last change in weights for momentum   
        self.ci = makeMatrix(self.ni, self.nh)
        self.co = makeMatrix(self.nh, self.no)

    def update(self, inputs):
        if len(inputs) != self.ni-1:
            raise ValueError('wrong number of inputs')

        # input activations
        for i in range(self.ni-1):
            #self.ai[i] = sigmoid(inputs[i])
            self.ai[i] = inputs[i]

        # hidden activations
        for j in range(self.nh):
            sum = 0.0
            for i in range(self.ni):
                sum = sum + self.ai[i] * self.wi[i][j]
            self.ah[j] = sigmoid(sum)

        # output activations
        for k in range(self.no):
            sum = 0.0
            for j in range(self.nh):
                sum = sum + self.ah[j] * self.wo[j][k]
            self.ao[k] = sigmoid(sum)

        return self.ao[:]


    def backPropagate(self, targets, N, M):
        if len(targets) != self.no:
            raise ValueError('wrong number of target values')

        # calculate error terms for output
        output_deltas = [0.0] * self.no
        for k in range(self.no):
            error = targets[k]-self.ao[k]
            output_deltas[k] = dsigmoid(self.ao[k]) * error

        # calculate error terms for hidden
        hidden_deltas = [0.0] * self.nh
        for j in range(self.nh):
            error = 0.0
            for k in range(self.no):
                error = error + output_deltas[k]*self.wo[j][k]
            hidden_deltas[j] = dsigmoid(self.ah[j]) * error

        # update output weights
        for j in range(self.nh):
            for k in range(self.no):
                change = output_deltas[k]*self.ah[j]
                self.wo[j][k] = self.wo[j][k] + N*change + M*self.co[j][k]
                self.co[j][k] = change
                #print N*change, M*self.co[j][k]

        # update input weights
        for i in range(self.ni):
            for j in range(self.nh):
                change = hidden_deltas[j]*self.ai[i]
                self.wi[i][j] = self.wi[i][j] + N*change + M*self.ci[i][j]
                self.ci[i][j] = change

        # calculate error
        error = 0.0
        for k in range(len(targets)):
            error = error + 0.5*(targets[k]-self.ao[k])**2
        return error


    def test(self, patterns):
        for p in patterns:
            print(p[0], '->', self.update(p[0]))

    def weights(self):
        print('Input weights:')
        for i in range(self.ni):
            print(self.wi[i])
        print()
        print('Output weights:')
        for j in range(self.nh):
            print(self.wo[j])

    def train(self, patterns, iterations=1000, N=0.5, M=0.1):
        # N: learning rate
        # M: momentum factor
        for i in range(iterations):
            error = 0.0
            for p in patterns:
                inputs = p[0]
                targets = p[1]
                self.update(inputs)
                error = error + self.backPropagate(targets, N, M)
            if i % 100 == 0:
                print('error %-.5f' % error)


def demo():
    # Teach network XOR function
    pat = [
        [[0,0], [0]],
        [[0,1], [1]],
        [[1,0], [1]],
        [[1,1], [0]]
    ]

    # create a network with two input, two hidden, and one output nodes
    n = NN(2, 2, 1)
    # train it with some patterns
    n.train(pat)
    # test it
    n.test(pat)



if __name__ == '__main__':
    demo()

error 0.94250
error 0.04287
error 0.00348
error 0.00164
error 0.00106
error 0.00078
error 0.00092
error 0.00053
error 0.00044
error 0.00038
[0, 0] -> [0.03036939032113823]
[0, 1] -> [0.9817636240847771]
[1, 0] -> [0.9816259907635363]
[1, 1] -> [-0.025585374843295334]
